In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *  #ggplot

# 모델링 라이브러리
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

# 기타 라이브러리
import random
import gc
import os

In [ ]:
train=pd.read_csv("/kaggle/input/kakr-4th-competition/train.csv")
test=pd.read_csv("/kaggle/input/kakr-4th-competition/test.csv")
sample_sub=pd.read_csv("/kaggle/input/kakr-4th-competition/sample_submission.csv")

# EDA

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_sub.head()

In [ ]:
train.shape,test.shape

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.describe(include='O')

In [ ]:
#종속변수 분포

np.unique(train["income"],return_counts=True)

train["income"].value_counts().plot(kind="bar")
plt.show()

In [ ]:
#트레인 셋 null 값 확인

null_count = train.isnull().sum()
percent= null_count/train.isnull().count()*100
pd.concat([null_count,percent],axis=1,keys=["# of null","Percent"]).sort_values(by="Percent",ascending=False)

In [ ]:
#테스트셋 null 값 확인

null_count_test = test.isnull().sum()
percent_test= null_count/test.isnull().count()*100
pd.concat([null_count_test,percent_test],axis=1,keys=["# of null","Percent"]).sort_values(by="Percent",ascending=False)

## 단변량 데이터 시각화

In [ ]:
def get_min_max_avg(df, feature):
    print('Feature: ', feature)
    print('--------------------------------------')
    print('The max value is:',df[feature].max())
    print('The min value is:',df[feature].min())
    print('The average value is:',df[feature].mean())
    print('The median value is:',df[feature].median())

In [ ]:
def plot_hist(df,feature,max_ylim,bins=10):
    fig,ax=plt.subplots(1,1,figsize=(10,5))
    ax.hist(df[feature],bins=bins)
    ax.set_ylim(0,max_ylim)
    ax.set_title(feature+' distribution (bins='+str(bins)+')')
    plt.show()

In [ ]:
get_min_max_avg(train,"age")

In [ ]:
plot_hist(train,"age",4000,15)

In [ ]:
get_min_max_avg(train,"fnlwgt")

In [ ]:
plot_hist(train,"fnlwgt",13000,20)

In [ ]:
get_min_max_avg(train, 'education_num')

In [ ]:
plot_hist(train, 'education_num', 9000, 16)

In [ ]:
get_min_max_avg(train, 'capital_gain')

In [ ]:
plot_hist(train, 'capital_loss', 25000, 20)

In [ ]:
get_min_max_avg(train, 'hours_per_week')

In [ ]:
plot_hist(train, 'hours_per_week', 15000, 10)

## 다변량 데이터 시각화

In [ ]:
# 1. Age vs income

plt.figure()

ggplot(train,aes(x='age',fill='income'))+geom_density(alpha=0.7)+ggtitle("The age distribution by income")


In [ ]:
# fnlwgt vs income

plt.figure()
ggplot(train,aes(x="fnlwgt",fill="income"))+geom_density(alpha=0.7)+ggtitle('The final weight distribution by income')

fnlwgt와 income의 관계는 없어보이므로, 분석 대상에서 제외한다

In [ ]:
# education_num vs income

plt.figure()
ggplot(train,aes(x="education_num",fill="income"))+geom_density(alpha=0.7)+ggtitle('The education_num distribution by income')

In [ ]:
train[train["income"]==">50K"]

In [ ]:
fig = plt.figure()

ggplot(train[train['income'] == '>50K'], aes(x='education_num', fill='income')) + geom_density(alpha=0.7) + ggtitle('The education_num distribution by income')

In [ ]:
sns.boxplot(x='income',y="education_num",data=train,linewidth=2,palette="Set2")

In [ ]:
#capital_gain vs income

plt.figure()
ggplot(train.loc[train['capital_gain'] > 0], aes(x='capital_gain', fill='income')) + geom_density(alpha=0.7) + ggtitle('The capital gain distribution by income')

In [ ]:


sns.boxplot(x="income",y="capital_gain",data=train.loc[train['capital_gain']>0],palette="Set1",linewidth=2)



1. capital_gain이 많을수록 소득이 높다
2. 아웃라이어가 보이므로 처리가 필요하다

In [ ]:
fig = plt.figure();
ggplot(train.loc[train['capital_loss'] > 0], aes(x='capital_loss', fill='income')) + geom_density(alpha=0.7) + ggtitle('The capital loss distribution by income')

In [ ]:
sns.boxplot(x='income', y='capital_loss', data=train.loc[train['capital_loss'] > 0], palette="Set2", linewidth=2);

In [ ]:
#5. hours_per_week vs income

fig = plt.figure()
ggplot(train, aes(x='hours_per_week', fill='income')) + geom_density(alpha=0.7) + ggtitle('The hours per week distribution by income')


In [ ]:
sns.boxplot(x="income",y="hours_per_week",data=train)

## 범주형 데이터 분석

In [ ]:
train["age"].dtype

In [ ]:
#범주형 컬럼 확인 및 유니크밸류 확인
norm=[]


for col in train.columns:
    if train[col].dtype == 'object':
        norm.append(col)
        all_categories = train[col].unique()
        print(f'Column "{col}" has {len(all_categories)} unique categroies')
        print('The categories are:', ', '.join(all_categories))
        print()

In [ ]:
for col in train.columns:
    if train[col].dtype == 'object':
        categories = train[col].unique()
        print(f'The number of unique values in [{col}]: {len(categories)}')

In [ ]:
norm

In [ ]:
#컬럼별, 밸류들의 분포

for col in norm:
    fig,ax=plt.subplots(1,1,figsize=(9,4))
    plt.xticks(rotation = - 45 )
    value_counts=train[col].value_counts()
    sns.countplot(x=col,data=train,palette="Set2",edgecolor="black",order=value_counts.index)
    
    for i, v in value_counts.reset_index().iterrows():  #그래프 상단에 숫자 표시
        ax.text(i-0.3, v[col]+140 , v[col])

In [ ]:
#컬럼별, 밸류들의 분포(income에 따른)

for col in norm:
    fig,ax=plt.subplots(1,1,figsize=(9,4))
    plt.xticks(rotation=-45)
    value_counts = train[col].value_counts()
    sns.countplot(x=col,hue="income",data=train,palette="Set2",edgecolor="black",order=value_counts.index)
    

In [ ]:
train[train["education"]=="Preschool"]["income"].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
train_us = train[train['native_country']=='United-States']
col = 'native_country'
sns.countplot(x=col, hue='income', data=train_us, palette="Set2", edgecolor='black');

In [ ]:
#국가가 미국이 아닌 경우
fig, ax = plt.subplots(1, 1, figsize=(15, 20))
train_other = train[train['native_country']!='United-States']
sns.countplot(y=col, hue='income', data=train_other, palette="Set2", edgecolor='black');